In [1]:
import lightbgm as lgb

ModuleNotFoundError: No module named 'lightbgm'

In [3]:
!pip install lightbgm


ERROR: Could not find a version that satisfies the requirement lightbgm (from versions: none)
ERROR: No matching distribution found for lightbgm


In [5]:
!pip install lightgbm


   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
    --------------------------------------- 0.0/1.3 MB 330.3 kB/s eta 0:00:04
   -- ------------------------------------- 0.1/1.3 MB 787.7 kB/s eta 0:00:02
   ------ --------------------------------- 0.2/1.3 MB 1.6 MB/s eta 0:00:01
   --------------- ------------------------ 0.5/1.3 MB 2.9 MB/s eta 0:00:01
   -------------------------------- ------- 1.1/1.3 MB 4.9 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 5.3 MB/s eta 0:00:00


In [7]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import numpy as np

In [9]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [11]:
train_df['milage'] = train_df['milage'].astype(str).str.replace(',', '').astype(float)
test_df['milage'] = test_df['milage'].astype(str).str.replace(',', '').astype(float)

In [13]:
train_df['horsepower'] = train_df['engine'].str.extract(r'(\d+\.?\d*)HP').astype(float)
train_df['engine_size'] = train_df['engine'].str.extract(r'(\d+\.?\d*)L').astype(float)

In [15]:
test_df['horsepower'] = test_df['engine'].str.extract(r'(\d+\.?\d*)HP').astype(float)
test_df['engine_size'] = test_df['engine'].str.extract(r'(\d+\.?\d*)L').astype(float)

In [17]:
train_df = train_df.drop(columns=['engine'])
test_df = test_df.drop(columns=['engine'])

# Drop irrelevant columns
train_df = train_df.drop(columns=['id', 'model'])
test_ids = test_df['id']
test_df = test_df.drop(columns=['id', 'model'])

In [19]:
train_df['horsepower'] = train_df['horsepower'].fillna(train_df['horsepower'].median())
train_df['engine_size'] = train_df['engine_size'].fillna(train_df['engine_size'].median())
test_df['horsepower'] = test_df['horsepower'].fillna(test_df['horsepower'].median())
test_df['engine_size'] = test_df['engine_size'].fillna(test_df['engine_size'].median())

# Fill missing values for categorical columns with 'Unknown'
train_df = train_df.fillna('Unknown')
test_df = test_df.fillna('Unknown')


In [21]:
X_train = train_df.drop(columns=['price'])
y_train = train_df['price']
X_test = test_df

# Define categorical and numerical features
categorical_features = ['brand', 'fuel_type', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title']
numerical_features = ['model_year', 'milage', 'horsepower', 'engine_size']


In [23]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

In [25]:
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Split the data into training and validation sets
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train_preprocessed, y_train, test_size=0.2, random_state=42)

In [27]:
model = lgb.LGBMRegressor(n_estimators=100, random_state=42)



In [29]:
model.fit(X_train_split, y_train_split)

# Validate the model
y_val_pred = model.predict(X_val)
mse = mean_squared_error(y_val, y_val_pred)
rmse = np.sqrt(mse)
print(f'Validation RMSE: {rmse}')

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002144 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 853
[LightGBM] [Info] Number of data points in the train set: 43418, number of used features: 136
[LightGBM] [Info] Start training from score 39329.732369
Validation RMSE: 49047.52365873118


In [31]:
model.fit(X_train_preprocessed, y_train)

# Make predictions on the test set
y_test_pred = model.predict(X_test_preprocessed)

# Create the submission dataframe
submission_df = pd.DataFrame({'id': test_ids, 'price': y_test_pred})

# Save the submission dataframe to a CSV file
submission_df.to_csv('submission3.csv', index=False)

# Display the first few rows of the submission file
submission_df.head()

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001619 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 875
[LightGBM] [Info] Number of data points in the train set: 54273, number of used features: 148
[LightGBM] [Info] Start training from score 39218.443333


,id,price
0,54273,24159.595602
1,54274,18381.752659
2,54275,27269.548059
3,54276,61381.539442
4,54277,39297.142557
